In [ ]:
import numpy as np
import pandas as pd
import datetime
import re

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Преобразование исходных данных

In [ ]:
"""Опишем функции для работы с исходным файлом"""

def convdate(string):
    """
    Конвертирует дату-строку в дату 
    """
    L = re.split('\.|:| ', string)
    L[0], L[2] = L[2], L[0]
    return datetime.datetime(*list(map(int, L)))

def dict_creation(file_name, start, finish):
    """
    Создаёт словарь: тик - стакан
    """
    D = dict()
    with open(file_name) as f:
        while True:
            line = f.readline().rstrip()
            if line[:8] == "Received":
                if start <= convdate(line[10:]) <= finish:
                    templob = [f.readline().rstrip() for _ in range(40)]
                    D[convdate(line[10:])] = templob
                elif convdate(line[10:]) > finish:
                    break
    return D


In [ ]:
"""Считаем файл"""

start = datetime.datetime(2020, 12, 2, 10)
finish = datetime.datetime(2020, 12, 2, 18, 40)

D = dict_creation("SBER.txt", start, finish)

In [ ]:
res_vector = []
for item in D:
    prices = []
    volumes = []
    vector = []
    for i in range(40):
        price, volume = D[item][i].split(';')[1:]
        prices.append(float(price))
        volumes.append(int(volume))
#    demo_volume = volumes[:20]
#    demo_ask = prices[:20]
#    for i in range(20):
#        prices.pop(0)
#        volumes.pop(0)
#    prices = demo_ask[::-1] + prices
#    volumes = demo_volume[::-1] + volumes
    prices = prices[::-1]
    volumes = volumes[::-1]
    for i in range(40):
        vector.append(prices[i])
        vector.append(volumes[i])
    res_vector.append(vector)

In [ ]:
indices = list(D.keys())
A = np.array(res_vector)
df = pd.DataFrame(A, index=indices)

Таким образом, мы получили матрицу Numpy и таблицу DataFrame, в которых строки - тики, а столбцы: в нечётных - цена по возрастанию, а в чётных столбцах соответствующий объём для цены из предыдущего столбца.

In [ ]:
df.head()

In [ ]:
# Ask already followed natural order
dfAskPrices = df.loc[:, range(40, 80, 2)]
dfAskVolumes = df.loc[:, range(41, 80, 2)]

dfBidPrices = df.loc[:, range(0, 40, 2)]
dfBidVolumes = df.loc[:, range(1, 40, 2)]

# Reverse Bid price and volumnes to make them follow natural order
#dfBidVolumes = dfBidVolumes[dfBidVolumes.columns[::-1]]

# Concatenate Bid and Ask together to form complete orderbook picture
dfPrices = df.iloc[:, range(0, 80, 2)]
dfVolumnes = df.iloc[:, range(1, 80, 2)]

#Rename columns starting from 1->20
dfPrices.columns = range(1, 41)
dfVolumnes.columns = range(1, 41)

dfPrices.head()

In [ ]:
dfVolumnes.head()

# Динамика цены акций на всех 20 уровнях со стороны продажи и со стороны покупки

In [ ]:
"""Графики не должны пересекаться, так как каждая кривая представляет собой динамику цены акции на ценовом уровне"""
fig = go.Figure()

#for i in dfPrices.columns:
# for i in range(1,2): 
#    fig.add_trace(go.Scatter(y=dfPrices[100:200][i]))
    
"""Если мы хотим использовать только 2 цвета. Один цвет - со стороны покупки, другой - со стороны продажи"""
for i in dfAskPrices.columns:
    fig.add_trace(go.Scatter(y=dfAskPrices[100:200][i],
                  line=dict(color='crimson')))
    
for i in dfBidPrices.columns:
    fig.add_trace(go.Scatter(y=dfBidPrices[100:200][i],
                  line=dict(color='lightslategrey')))

fig.update_layout(
    title='40 ценовых уровней в книге заявок',
    xaxis_title="Время",
    yaxis_title="Цена",
#     template='plotly_dark',
    height=500,
    showlegend=False,
)

fig.show()

# Объёмы ценовых уровней для тиков

Один цвет соответствует одному тику. По горизонтали отложен объём в лотах. На представленной диаграмме видим, что на 11ом ценовом уровне только в начальный момент времени, во время первого тика было около 12 тысяч лотов, затем позиция исчезла.

In [ ]:
fig = px.bar(dfVolumnes.head(5).transpose(), orientation='h')
fig.show()

# Состояние книги заказов в заданный момент времени (тик)

In [ ]:
colors = ['lightslategrey',] * 20
colors = colors + ['crimson',] * 20

In [ ]:
fig = go.Figure()
timestamp = 10000

fig.add_trace(go.Bar(
    y= ['{:.2f}'.format(x) for x in dfPrices[:timestamp].values[1].tolist()],
    x=dfVolumnes[:timestamp].values[1].tolist(),
    orientation='h',
    marker_color=colors
))

fig.update_layout(
    title='40 ценовых уровней, размер бара представляет объём в лотах',
    xaxis_title="Объём, лот",
    yaxis_title="Цена",
#     template='plotly_dark'
)

fig.show()

# Изображение на одной диаграмме для нескольких тиков

In [ ]:
fig = make_subplots(rows=1, cols=2)

#for i in dfPrices.columns: 
#    fig.add_trace(go.Scatter(y=dfPrices.head(20)[i]), row=1, col=1)
"""Если мы хотим использовать только 2 цвета. Один цвет - со стороны покупки, другой - со стороны продажи"""
for i in dfAskPrices.columns:
    fig.add_trace(go.Scatter(y=dfAskPrices[100:200][i],
                  line=dict(color='crimson')))
    
for i in dfBidPrices.columns:
    fig.add_trace(go.Scatter(y=dfBidPrices[100:200][i],
                  line=dict(color='lightslategrey')))

timestamp = 50000

fig.add_trace(go.Bar(
    y= ['{:.2f}'.format(x) for x in dfPrices[:timestamp].values[0].tolist()],
    x=dfVolumnes[:timestamp].values[0].tolist(),
    orientation='h',
    marker_color=colors
), row=1, col=2)

fig.update_layout(
    title='Динамика 40 ценовых уровней в книге заявок для нескольких тиков, справа - начальное состояние',
    xaxis_title="Время",
    yaxis_title="Цена",
    template='plotly_dark'
)

fig.show()

# Анимация динамики цены акции

In [ ]:
widthOfTime = 100;

fig = go.Figure(
    data=[go.Scatter(x=dfPrices.index[:widthOfTime].tolist(), y=dfPrices[:widthOfTime][1].tolist(),
                     name="frame",
                     mode="lines",
                     line=dict(width=2, color="crimson")),
          ],
    layout=go.Layout(width=1000, height=400,
#                      xaxis=dict(range=[0, 100], autorange=False, zeroline=False),
#                      yaxis=dict(range=[0, 1], autorange=False, zeroline=False),
                     title="Динамика цены акции",
                     xaxis_title="Время",
                     yaxis_title="Цена",
                     template='plotly_dark',
                     hovermode="closest",
                     updatemenus=[dict(type="buttons",
                                       showactive=True,
                                       x=0.01,
                                       xanchor="left",
                                       y=1.15,
                                       yanchor="top",
                                       font={"color":'blue'},
                                       buttons=[dict(label="Play",
                                                     method="animate",
                                                     args=[None])])]),

    frames=[go.Frame(
        data=[go.Scatter(
            x=dfPrices.iloc[k:k+widthOfTime].index.tolist(),
            y=dfPrices.iloc[k:k+widthOfTime][1].tolist(),
            mode="lines",
            line=dict(color="red", width=2))
        ]) for k in range(widthOfTime, 1000)]
)

fig.show()

# Анимация объема книги заказов во времени 

In [ ]:
timeStampStart = 100


fig = go.Figure(
    data=[go.Bar(y= ['{:.2f}'.format(x) for x in dfPrices[:timeStampStart].values[0].tolist()],
                 x=dfVolumnes[:timeStampStart].values[0].tolist(),
                 orientation='h',
                 name="priceBar",
                 marker_color=colors),
          ],
    layout=go.Layout(width=800, height=450,
                     title="Динамика состояния книги заявок",
                     xaxis_title="Объём, лот",
                     yaxis_title="Цена",
                     template='plotly_dark',
                     hovermode="closest",
                     updatemenus=[dict(type="buttons",
                                       showactive=True,
                                       x=0.01,
                                       xanchor="left",
                                       y=1.15,
                                       yanchor="top",
                                       font={"color":'blue'},
                                       buttons=[dict(label="Play",
                                                     method="animate",
                                                     args=[None])])]),
    frames=[go.Frame(
        data=[go.Bar(y= ['{:.2f}'.format(x) for x in dfPrices.iloc[k].values.tolist()],
                     x=dfVolumnes.iloc[k].values.tolist(),
                     orientation='h',
                     marker_color=colors)],
        layout=go.Layout(width=800, height=450,
                     title="Динамика книги заявок [Время=" + str(indices[k]) +"]",
                     xaxis_title="Объём, лот",
                     yaxis_title="Цена",
                     template='plotly_dark',
                     hovermode="closest")) for k in range(timeStampStart, 500)]
)

fig.show()